Inspired by `visual law` theme, I decided to explore the #ChatGPT and #DALL·E services to see how they can be applied in this context. The idea is to use these services to generate images that visually represent the content of legal texts.

Although the images generated by the services are not as clear and precise as we would like, it is possible to make a clear association between them and the texts. In the video accompanying this repository, I demonstrate how I used ChatGPT and DALL·E to generate text summaries and then use these summaries to generate corresponding images.

This project is a great way to explore the possibilities of artificial intelligence in the legal and visual context. Through the use of these services, we can automate the process of creating images that visually represent the content of complex texts.

#| label: app 
![App Flow Diagram](app.png)

The tough part of api was done by OpenAI. I just used their API to generate text and images. The code is available in the `api.py` file bellow.

```python
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import openai
import os
from bs4 import BeautifulSoup

from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

app = FastAPI()

# Allow cross-origin requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)


class SummaryRequest(BaseModel):
    text: str

@app.post("/summary")
async def summarize(request: SummaryRequest):
    soup = BeautifulSoup(request.text, 'html.parser')
    text = soup.get_text()
    content = f'A partir do pedido principal do texto a seguir, descreva em ingles com limite de 128 caracteres um cenario ou um personagem. Ignore nomes proprios!\n\n{text}'
    #print(content)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[
                {"role": "user", "content": content}])
    summary = response.choices[0].message.content.strip()
    print(f'summary: {summary}')

    try:
        image_response = openai.Image.create(
            prompt=summary,
            size="512x512"
        )
        print(image_response)
        image_url = image_response['data'][0]['url']
    except Exception as e:
        print("Exception message:", str(e))
        image_url = "https://via.placeholder.com/512x512.png?text=Image+Not+Available"
        return {"image_url": image_url}    
    return {"image_url": image_url}    
```

After sending the following prompt to the ChatGPT service, I received the following summary:
```python
'INTERVALO PARA RECUPERAÇÃO TÉRMICA DO EMPREGADO. AMBIENTE ARTIFICIALMENTE FRIO. HORAS EXTRAS. ART. 253 DA CLT. APLICAÇÃO ANALÓGICA – Res. 185/2012, DEJT divulgado em 25, 26 e 27.09.2012 O empregado submetido a trabalho contínuo em ambiente artificialmente frio, nos termos do parágrafo único do art. 253 da CLT, ainda que não labore em câmara frigorífica, tem direito ao intervalo intrajornada previsto no caput do art. 253 da CLT.'
```

```python
'An employee working in an artificially cold environment has the right to an interval, as per article 253 of CLT. They are entitled to recover their thermal equilibrium.'
```

With this summary, I was able to generate the following image:

#| label: dalle 
![DALL·E Image](2023-04-02-09-39-29.png)

The whole app code, including the react UX, is avaliable at [GitHub](https://github.com/fabraz/visuallaw)

Enjoy!

FB